In [5]:
import pandas as pd 
import numpy as np
import quandl
import datetime
import math



In [6]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import median_absolute_error

from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score

# https://scikit-learn.org/stable/modules/model_evaluation.html Score pour regression !!!

In [7]:
#Recap fait que les Errors et donc pas des score (les erreur le plus bas le mieux les score le plus haut le mieux)

price_score = pd.DataFrame(index =["AAE","MSE","MSLE","MAE"],columns=['Kalman', 'ARIMA', 'LSTM', 'RF', 'SVM']).fillna(0)
price_score

,Kalman,ARIMA,LSTM,RF,SVM
AAE,0,0,0,0,0
MSE,0,0,0,0,0
MSLE,0,0,0,0,0
MAE,0,0,0,0,0


In [8]:
#on garde que le MSE mais sur différent jeux de donné
df_score = pd.DataFrame(index =["Price","Volatility","Log Return","Simulate ?"],columns=['Kalman', 'ARIMA', 'LSTM', 'RF', 'SCM']).fillna(0)
Kalman_score=[]
ARIMA_score=[]
LSTM_score=[]
RF_score=[]
SVM_score=[]

df_score

,Kalman,ARIMA,LSTM,RF,SCM
Price,0,0,0,0,0
Volatility,0,0,0,0,0
Log Return,0,0,0,0,0
Simulate ?,0,0,0,0,0


In [9]:
Data=pd.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data_quandl\Data_total.csv")
#Data_std=pd.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data_quandl\Data_std_total.csv")
#Data_logreturn=pd.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data_quandl\Data_logr_total.csv")



# Fonction pour calculer tous les score

In [133]:
def clean(df,logo):
    logo2=logo.copy()
    for l in logo:
        if l not in df.columns:
            logo2.remove(l)
    return logo2
            
def normalisation(df,logo2,nom_methode):
    logo=clean(df,logo2)
    k=logo[0]
    m, sigma,= df[k].mean(), df[k].std(),       
    df_n=df[[k,k+'_'+nom_methode]].apply(lambda x : (x-m)/sigma)
    for l in logo[1:]:
        m, sigma,= df[l].mean(), df[l].std(),
        df_i=df[[l,l+'_'+nom_methode]].apply(lambda x : (x-m)/sigma)
        df_n=df_n.join(df_i)
    return df_n


def calcul_score(df, logo,nom_methode):
    df_n=normalisation(df,logo,nom_methode)
    trues=[]
    predictions=[]
    for p in logo:
        try:
            l=p+'_'+nom_methode
            trues += df_n[p].tolist()
            predictions += df_n[l].tolist() #que chaque pred bien de la forme LOGO_Méthode 
        except:
            pass
    return mean_squared_error(trues,predictions)


###########" Avec l'ancienne normalisation" #############"
def normalisation2(df,logo2,nom_methode):
    logo=clean(df,logo2)
    df_n=pd.DataFrame()
    for l in logo:
        l2=l+'_'+nom_methode
        df_n[l]=df[l]/df[l]
        df_n[l2]=df[l2]/df[l]
    return df_n

def calcul_score2(df, logo,nom_methode):
    df_n=normalisation2(df,logo,nom_methode)
    trues=[]
    predictions=[]
    for p in logo:
        try:
            l=p+'_'+nom_methode
            trues += df_n[p].tolist()
            predictions += df_n[l].tolist() #que chaque pred bien de la forme LOGO_Méthode 
        except:
            pass
    return mean_squared_error(trues,predictions)
    
    

# Chargement des données

In [15]:
df_cie=pd.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data_quandl\Liste_entreprise.csv")
df_cie=df_cie.iloc[:, 1:]

logo=df_cie['logo'].tolist()
df_cie.head(2)

,logo,logo_pred
0,AAPL,AAPL_Pred
1,ABT,ABT_Pred


In [108]:
Kalman_pred=pd.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data_quandl\Kalman_All_and_Pred.csv")
Kalman_pred=Kalman_pred.set_index('Date')

#Kalman_pred_std=pd.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data_quandl\Kalman_All_and_Pred_std.csv")
#Kalman_pred_std=Kalman_pred_std.set_index('Date')

#Kalman_pred_logr=pd.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data_quandl\Kalman_All_and_Pred_logr.csv")
#Kalman_pred_logr=Kalman_pred_logr.set_index('Date')

'AMD' not in ARIMA_pred.columns

True

In [70]:
ARIMA_pred=pd.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data_quandl\ARIMA_All_and_Pred.csv")
ARIMA_pred=ARIMA_pred.set_index('Date')


In [72]:
Lstm_pred=pd.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data_quandl\LSTM_All_and_Pred.csv")
Lstm_pred=Lstm_pred.set_index('Date')

In [135]:
calcul_score(Kalman_pred,logo,'Pred'), calcul_score(ARIMA_pred,logo,'ARIMA'), calcul_score(Lstm_pred,logo,'Pred') , calcul_score2(Kalman_pred,logo,'Pred'), calcul_score2(ARIMA_pred,logo,'ARIMA'), calcul_score2(Lstm_pred,logo,'Pred')




(6.018855477072234,
 1.3812040816253552,
 13.163868162771063,
 0.004105760432481327,
 0.0005937677789747236,
 0.005637191917386555)

# Pour calculer toutes les erreurs pour toutes les méthodes

# KALMAN

In [119]:
#Normalisation avec Pred en % du True

Kalman_pred_n=pd.DataFrame()
for l in logo:
    l2=l+'_Pred'
    Kalman_pred_n[l]=Kalman_pred[l]/Kalman_pred[l]
    Kalman_pred_n[l2]=Kalman_pred[l2]/Kalman_pred[l]

In [120]:
Kalman_pred_n #pour normaliser faire plutôt une "vrai normalisation" X-m/sigma (m et sigma du jeux de donné jusqu'au 5 janvier et pas le 8)

,AAPL,AAPL_Pred,ABT,ABT_Pred,ACN,ACN_Pred,ATVI,ATVI_Pred,ADBE,ADBE_Pred,...,AAL,AAL_Pred,BLK,BLK_Pred,CBS,CBS_Pred,MSFT,MSFT_Pred,FB,FB_Pred
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-06,1.0,1.075931,1.0,1.044450,1.0,1.023102,1.0,1.038210,1.0,1.019687,...,1.0,1.029543,1.0,1.022956,1.0,1.023099,1.0,1.020086,1.0,1.016454
2016-01-07,1.0,1.111608,1.0,1.063429,1.0,1.048913,1.0,1.048044,1.0,1.037591,...,1.0,1.044697,1.0,1.063078,1.0,1.037032,1.0,1.051438,1.0,1.062324
2016-01-08,1.0,1.095701,1.0,1.077980,1.0,1.053539,1.0,1.058345,1.0,1.047481,...,1.0,1.042318,1.0,1.081925,1.0,1.010888,1.0,1.043636,1.0,1.062222


In [122]:
Kpn=Kalman_pred_n

df_score_kalman=pd.DataFrame(columns=['Companie','Mean AE','Mean SE','Mean LSE','Median AE'])


trues=[]
predictions=[]

for l in logo:
    for k in Kpn[l+'_Pred']:
        trues.append(1)
        predictions.append(k)
        
TSAAE = mean_absolute_error(trues,predictions)
TSMSE = mean_squared_error(trues,predictions)
TSMLSE= mean_squared_log_error(trues,predictions)
TSMAE = median_absolute_error(trues,predictions)

Score_kalman= [TSAAE, TSMSE, TSMLSE, TSMAE]
Score_kalman  

[0.05308333151957322,
 0.004105760432481327,
 0.0009736831012384756,
 0.04457355467622548]

In [77]:
price_score['Kalman']=Score_kalman
price_score*100

,Kalman,ARIMA,LSTM,RF,SCM
AAE,5.308333,0,6.194613,0,0
MSE,0.410576,0,0.460283,0,0
MSLE,0.097368,0,0.110599,0,0
MAE,4.457355,0,6.306660,0,0


# ARIMA

In [79]:
#Normalisation avec Pred en % du True

ARIMA_pred_n=pd.DataFrame()

for l in logo:
    try:
        l2=l+'_ARIMA'
        ARIMA_pred_n[l]=ARIMA_pred[l]/ARIMA_pred[l]
        ARIMA_pred_n[l2]=ARIMA_pred[l2]/ARIMA_pred[l]
       # ARIMA_pred_n[l+'_Ecart']=abs(ARIMA_pred[l]-ARIMA_pred[l2])
       # ARIMA_pred_n[l+'_Ecart_Carré']=(ARIMA_pred[l]-ARIMA_pred[l2])**2
    except:
        pass    


In [80]:
ARIMA_pred_n

,AAPL,AAPL_ARIMA,ABT,ABT_ARIMA,ACN,ACN_ARIMA,ATVI,ATVI_ARIMA,ADBE,ADBE_ARIMA,...,AAL,AAL_ARIMA,BLK,BLK_ARIMA,CBS,CBS_ARIMA,MSFT,MSFT_ARIMA,FB,FB_ARIMA
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-06,1.0,1.021345,1.0,1.012801,1.0,1.008709,1.0,1.017248,1.0,1.018470,...,1.0,0.979203,1.0,1.012402,1.0,1.000295,1.0,1.020406,1.0,1.004899
2016-01-07,1.0,1.047155,1.0,1.030537,1.0,1.034452,1.0,1.023293,1.0,1.025335,...,1.0,1.022079,1.0,1.044991,1.0,1.015816,1.0,1.036193,1.0,1.057753
2016-01-08,1.0,0.997380,1.0,1.022318,1.0,1.015683,1.0,1.019786,1.0,1.017983,...,1.0,0.998049,1.0,1.025591,1.0,0.974842,1.0,0.995309,1.0,1.002205


In [81]:
Apn=ARIMA_pred_n

df_score_kalman=pd.DataFrame(columns=['Companie','Mean AE','Mean SE','Mean LSE','Median AE'])


trues=[]
predictions=[]

for p in logo:
    try:
        l=p+'_ARIMA'
        for k in Apn[l]:
            trues.append(1)
            predictions.append(k)
    except:
        pass
        
TSAAE = mean_absolute_error(trues,predictions)
TSMSE = mean_squared_error(trues,predictions)
TSMLSE= mean_squared_log_error(trues,predictions)
TSMAE = median_absolute_error(trues,predictions)

Score_ARIMA= [TSAAE, TSMSE, TSMLSE, TSMAE]
Score_ARIMA  

[0.0199111963524565,
 0.0005937677789747236,
 0.0001459038195184978,
 0.018469500061139232]

In [82]:
price_score['ARIMA']=Score_ARIMA
price_score*100

,Kalman,ARIMA,LSTM,RF,SCM
AAE,5.308333,1.991120,6.194613,0,0
MSE,0.410576,0.059377,0.460283,0,0
MSLE,0.097368,0.014590,0.110599,0,0
MAE,4.457355,1.846950,6.306660,0,0


# LSTM

##               LSTM Price 

In [124]:
#Normalisation avec Pred en % du True

Lstm_pred_n=pd.DataFrame()
for l in logo:
    l2=l+'_Pred'
    Lstm_pred_n[l]=Lstm_pred[l]/Lstm_pred[l]
    Lstm_pred_n[l2]=Lstm_pred[l2]/Lstm_pred[l]
    Lstm_pred_n[l+'_Ecart']=abs(Lstm_pred_n[l]-Lstm_pred_n[l2])
    Lstm_pred_n[l+'_Ecart_Carré']=(Lstm_pred_n[l]-Lstm_pred_n[l2])**2



In [125]:
Lstm_pred_n[['AAPL','AAPL_Pred']]

,AAPL,AAPL_Pred
Date,,
2016-01-06,1.0,1.055950
2016-01-07,1.0,1.091931
2016-01-08,1.0,1.121563


####  Calcul des scores

In [131]:
Lpn=Lstm_pred_n
df_score_lstm=pd.DataFrame(columns=['Companie','Mean AE','Mean SE','Mean LSE','Median AE'])

trues=[]
predictions=[]

for l in logo:
    for k in Lpn[l+'_Pred']:
        trues.append(1)
        predictions.append(k)
        
TSAAE = mean_absolute_error(trues,predictions)
TSMSE = mean_squared_error(trues,predictions)
TSMLSE= mean_squared_log_error(trues,predictions)
TSMAE = median_absolute_error(trues,predictions)

Score_lstm= [TSAAE, TSMSE, TSMLSE, TSMAE]
Score_lstm  

[0.0658468180645321,
 0.005637191917386555,
 0.0013490339895733934,
 0.06759237687977027]

In [132]:
price_score['LSTM']=Score_lstm
price_score*100

,Kalman,ARIMA,LSTM,RF,SVM
AAE,0,0,6.584682,0,0
MSE,0,0,0.563719,0,0
MSLE,0,0,0.134903,0,0
MAE,0,0,6.759238,0,0


In [2]:
#SVM pas SCM, méthode qui décalle que de un jour
#Faire une ligne volatilité une price et une return (pas besoin de prendre 4 error mais en selectionner une -MSE)